In [1]:
import re

# Define token types
TOKEN_SPEC = [
    ('PRINT', r'print'),          # Print keyword
    ('NUMBER', r'\d+'),           # Integer
    ('IDENTIFIER', r'[a-zA-Z_]\w*'),  # Variable names
    ('ASSIGN', r'='),             # Assignment operator
    ('PLUS', r'\+'),              # Addition
    ('MINUS', r'-'),              # Subtraction
    ('MULTIPLY', r'\*'),          # Multiplication
    ('DIVIDE', r'/'),             # Division
    ('MODULUS', r'%'),            # Modulus
    ('LPAREN', r'\('),            # Left parenthesis
    ('RPAREN', r'\)'),            # Right parenthesis
    ('SEMICOLON', r';'),          # End of statement
    ('SKIP', r'[ \t]+'),          # Skip spaces and tabs
    ('MISMATCH', r'.')            # Any other character
]

TOKEN_REGEX = '|'.join(f'(?P<{name}>{pattern})' for name, pattern in TOKEN_SPEC)

def tokenize(code):
    tokens = []
    position = 0
    for match in re.finditer(TOKEN_REGEX, code):
        kind = match.lastgroup
        value = match.group()
        if kind == 'SKIP':
            position += len(value)  # Skip whitespace but track position
            continue
        elif kind == 'MISMATCH':
            raise SyntaxError(f"Unexpected character: {value} at position {position}")
        tokens.append((kind, value))
        position += len(value)
    return tokens


In [2]:
class ASTNode:
    pass

class AssignNode(ASTNode):
    def __init__(self, name, value):
        self.name = name
        self.value = value

class BinOpNode(ASTNode):
    def __init__(self, left, op, right):
        self.left = left
        self.op = op
        self.right = right

class PrintNode(ASTNode):
    def __init__(self, value):
        self.value = value

class NumberNode(ASTNode):
    def __init__(self, value):
        self.value = value

class VariableNode(ASTNode):
    def __init__(self, name):
        self.name = name


In [3]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def consume(self, expected_type):
        if self.pos < len(self.tokens) and self.tokens[self.pos][0] == expected_type:
            token = self.tokens[self.pos]
            self.pos += 1
            return token
        if self.pos < len(self.tokens):
            token = self.tokens[self.pos]
            raise SyntaxError(f"Expected {expected_type}, but found {token[1]} at position {self.pos}")
        else:
            raise SyntaxError(f"Unexpected end of input; expected {expected_type}")

    def parse(self):
        statements = []
        while self.pos < len(self.tokens):
            if self.tokens[self.pos][0] == 'IDENTIFIER':
                statements.append(self.parse_assignment())
            elif self.tokens[self.pos][0] == 'PRINT':
                statements.append(self.parse_print())
            else:
                token = self.tokens[self.pos]
                raise SyntaxError(f"Unexpected token: {token[1]} at position {self.pos}")
        return statements

    def parse_assignment(self):
        var_name = self.consume('IDENTIFIER')[1]
        self.consume('ASSIGN')
        value = self.parse_expression()
        self.consume('SEMICOLON')
        return AssignNode(var_name, value)

    def parse_print(self):
        self.consume('PRINT')
        value = self.parse_expression()
        self.consume('SEMICOLON')
        return PrintNode(value)

    def parse_expression(self):
        if self.pos >= len(self.tokens):
            raise SyntaxError("Incomplete expression: unexpected end of input")
        left = self.parse_term()
        while self.pos < len(self.tokens) and self.tokens[self.pos][0] in ('PLUS', 'MINUS'):
            op = self.consume(self.tokens[self.pos][0])[0]
            if self.pos >= len(self.tokens) or self.tokens[self.pos][0] not in ('NUMBER', 'IDENTIFIER', 'LPAREN'):
                raise SyntaxError(f"Expected a term after operator {op}, but found {self.tokens[self.pos][1]} at position {self.pos}")
            right = self.parse_term()
            left = BinOpNode(left, op, right)
        return left

    def parse_term(self):
        left = self.parse_factor()
        while self.pos < len(self.tokens) and self.tokens[self.pos][0] in ('MULTIPLY', 'DIVIDE', 'MODULUS'):
            op = self.consume(self.tokens[self.pos][0])[0]
            if self.pos >= len(self.tokens) or self.tokens[self.pos][0] not in ('NUMBER', 'IDENTIFIER', 'LPAREN'):
                raise SyntaxError(f"Expected a term after operator {op}, but found {self.tokens[self.pos][1]} at position {self.pos}")
            right = self.parse_factor()
            left = BinOpNode(left, op, right)
        return left

    def parse_factor(self):
        if self.pos >= len(self.tokens):
            raise SyntaxError("Incomplete expression: expected a factor but found end of input")
        token = self.tokens[self.pos]
        if token[0] == 'NUMBER':
            self.consume('NUMBER')
            return NumberNode(int(token[1]))
        elif token[0] == 'IDENTIFIER':
            self.consume('IDENTIFIER')
            return VariableNode(token[1])
        elif token[0] == 'LPAREN':
            self.consume('LPAREN')
            expr = self.parse_expression()  # Parse the inner expression
            if self.pos >= len(self.tokens) or self.tokens[self.pos][0] != 'RPAREN':
                raise SyntaxError(f"Missing closing parenthesis for expression starting at position {self.pos - 1}")
            self.consume('RPAREN')
            return expr
        else:
            raise SyntaxError(f"Invalid factor: {token[1]} at position {self.pos}")


In [4]:
class Interpreter:
    def __init__(self):
        self.variables = {}

    def execute(self, nodes):
        for node in nodes:
            if isinstance(node, AssignNode):
                value = self.evaluate(node.value)
                self.variables[node.name] = value
            elif isinstance(node, PrintNode):
                result = self.evaluate(node.value)
                print(result)

    def evaluate(self, node):
        if isinstance(node, NumberNode):
            return node.value
        elif isinstance(node, VariableNode):
            value = self.variables.get(node.name)
            if value is None:
                raise NameError(f"Undefined variable: {node.name}")
            return value
        elif isinstance(node, BinOpNode):
            left = self.evaluate(node.left)
            right = self.evaluate(node.right)
            if node.op == 'PLUS':
                return left + right
            elif node.op == 'MINUS':
                return left - right
            elif node.op == 'MULTIPLY':
                return left * right
            elif node.op == 'DIVIDE':
                if right == 0:
                    raise ZeroDivisionError("Division by zero is not allowed")
                return left / right
            elif node.op == 'MODULUS':
                if right == 0:
                    raise ZeroDivisionError("Modulus by zero is not allowed")
                return left % right
            else:
                raise ValueError(f"Unsupported operator: {node.op}")
        else:
            raise ValueError(f"Unknown node type: {node}")

In [5]:
# Test Case 1: Basic Arithmetic

code = """
x = 10;
y = x + 5;
z = y * 2;
print(z);
"""
# Expected Output:
# 30


In [159]:
# Test Case 2: Division and Modulus

code = """
a = 20 / 5;
b = 23 % 5;
print(a);
print(b);
"""
# Expected Output:
# 4.0
# 3


In [161]:
# Test Case 3: Parentheses and Operator Precedence

code = """
x = (10 + 5) * (2 + 3);
y = x / (10 - 8);
print(x);
print(y);
"""
# Expected Output:
# 75
# 37.5


In [163]:
# Test Case 4: Undefined Variable

code = """
x = 10;
print(y);
"""
# Expected Error:
# NameError: Undefined variable: y


In [165]:
# Test Case 5: Division by Zero

code = """
x = 10 / 0;
print(x);
"""
# Expected Error:
# ZeroDivisionError: Division by zero is not allowed


In [167]:
# Test Case 6: Modulus by Zero

code = """
x = 10 % 0;
print(x);
"""
# Expected Error:
# ZeroDivisionError: Modulus by zero is not allowed


In [290]:
# Test Case 7: Nested Parentheses

code = """
x = ((10 + 2) * (5 - 3)) + (20 / (4 + 1));
print(x);
"""
# Expected Output:
# 28.0


In [173]:
# Test Case 8: Multiple Statements

code = """
x = 10;
y = 20;
z = x + y * 2;
print(x);
print(y);
print(z);
"""
# Expected Output:
# 10
# 20
# 50


In [246]:
# Test Case 9: Invalid Character

code = """
x = 10 @ 5;
"""
# Expected Error:
# SyntaxError: Unexpected character: @ at position 6


In [232]:
# Test Case 10: Incomplete Expression

code = """
x = (10 + ;
"""
# Expected Error:
# SyntaxError: Missing closing parenthesis for expression starting at position 4


In [9]:
# Test Case 11: Unexpected Token

code = """
x = 10 + ;
"""
# Expected Error:
# SyntaxError: Expected a term after operator +, but found ; at position 4


In [13]:
# Test Case 12: Complex Nested Operations

code = """
x = (10 + (5 * 3)) - ((20 / 4) + 2);
print(x);
"""
# Expected Output:
# 18.0


In [14]:
tokens = tokenize(code)
parser = Parser(tokens)
ast = parser.parse()
interpreter = Interpreter()
interpreter.execute(ast)

18.0
